© 2024 Luca Kunz. Commercial use is subject to the terms of the source repository's license. All other commercial rights are reserved.

# Compute TRAP translation speeds

We load the yearly TRAPS pkl files, stack them to one file and calculate the translation speeds $c_x$ and $c_y$ of every TRAP instance.  
Then, we export the 20-years dataframe and the yearly dataframes again. The yearly files will be further processed in the next script.

Since the computation is very time-consuming, we do not apply it in the actual tracking algorithm but in this separate script.  
Runtime ~14hours

In [1]:
import os
import sys
import numpy as np
from scipy.interpolate import interp2d
import pandas as pd
import time
import datetime
import pickle

# import metpy.calc as mpcalc
# from metpy.units import units

from IPython.display import display, Audio
# import jupyter notebook files like regular python modules
import import_ipynb
from aa_define_classes import TRAPSdata

importing Jupyter notebook from aa_define_classes.ipynb


In [2]:
# measure the computation time for the entire script
start_script_timer = time.perf_counter()

# Preliminary

## Pickle object saving function

In [3]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

## Files and paths

In [4]:
# if script is running in jupyter lab
if sys.argv[0].endswith("ipykernel_launcher.py"):
    # set the velocity product
    vel_product_ID = 1
    epsilon_ID = 1
    notebook_run = True
    # save_fig = True
    save_fig = False

    
# if script is running as python script
else:
    # read in product from bash
    vel_product_ID = int(sys.argv[1])
    # read in epsilon from bash
    epsilon_ID = int(sys.argv[2])
    notebook_run = False
    save_fig = True


vel_product_short = ['ENSRYS_24HM', 'MULTIOBS_24HI', 'MULTIOBS_24HM', 'SEALEVEL_24HI'][vel_product_ID]

vel_product_long = ['CMEMS GLOBAL_REANALYSIS_PHY_001_031 ENSEMBLE MEAN (1/4°, 24HM)', 
                    'CMEMS MULTIOBS_GLO_PHY_REP_015_004 (1/4°, 24HI)', 
                    'CMEMS MULTIOBS_GLO_PHY_REP_015_004 (1/4°, 24HM)', 
                    'CMEMS SEALEVEL_GLO_PHY_L4_MY_008_047 (1/4°, 24HI)'][vel_product_ID]


years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', 
         '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

epsilon_value = [0.1, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0][epsilon_ID]
epsilon_string = ['_e010', '_e025', '_e050', '_e075', '_e100', '_e125', '_e150', '_e175', '_e200'][epsilon_ID]

In [5]:
# define the path to the pkl files
pkl_TRAPS_importpath = 'export_pkl/' + vel_product_short + '/20XX/'
pkl_TRAPS_exportpath = 'export_pkl/' + vel_product_short + '/'

# Import TRAPS DataFrames

Load the yearly TRAPS dataframes and stack them together to an overall 2000-2019 dataframe.  
This is faster than loading the original 2000-2019 dataframe.  

In [6]:
# the lists to store the subdataframes to feed into pd.concat() after the loop
pd_TRAPS_sdfs = []

for year in years:
    
    start_timer = time.perf_counter()

    # define the pkl files to load
    pkl_TRAPS_importname = vel_product_short + epsilon_string + '_TRAPS_TRACKED_' + year + '.pkl'

    # load the pickle object for the current year
    with open(pkl_TRAPS_importpath + pkl_TRAPS_importname, 'rb') as inp:

        # cdf for current DataFrame, this will be overwritten every loop
        pd_TRAPS_sdfs.append(pickle.load(inp).pd_TRAPS_df.copy())

    stop_timer = time.perf_counter()
    print('loaded ' + pkl_TRAPS_importname + f' in: {stop_timer - start_timer:0.4f} seconds')

    
# stack the yearly dataframes
pd_TRAPS_df = pd.concat(pd_TRAPS_sdfs, copy=True)    
    
# reset the index after stacking
pd_TRAPS_df.reset_index(drop=True, inplace=True)

# save memory
del pd_TRAPS_sdfs

loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2000.pkl in: 2.3877 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2001.pkl in: 1.6612 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2002.pkl in: 1.8855 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2003.pkl in: 1.5765 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2004.pkl in: 1.7535 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2005.pkl in: 2.1036 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2006.pkl in: 2.1351 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2007.pkl in: 0.9940 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2008.pkl in: 2.3896 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2009.pkl in: 1.0151 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2010.pkl in: 2.6947 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2011.pkl in: 0.9821 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2012.pkl in: 1.0213 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2013.pkl in: 3.1061 seconds
loaded MULTIOBS_24HI_e025_TRAPS_TRACKED_2014.pkl

In [7]:
# print check
pd_TRAPS_df

,TRAP_ID,time,core_lon,core_lat,core_attraction,ED_TRAP,curve_lons,curve_lats,curve_attractions,origin_ID,lifetime,age,track_s1_peak
0,0,2000-01-01,-159.045688,23.400981,-0.625117,True,"[-159.465202832297, -159.40044405039814, -159....","[23.1889083623293, 23.241294361583048, 23.2868...","[-0.31061579015898855, -0.4048254423681092, -0...",200001010000 000,16,1,-0.650914
1,1,2000-01-01,-159.503564,24.127516,-0.298961,True,"[-159.159668021201, -159.23795868926015, -159....","[23.8158077645795, 23.84382718973737, 23.89130...","[-0.30928585683127613, -0.24813203636879133, -...",200001010000 001,1,1,-0.298961
2,2,2000-01-01,-157.098753,23.351263,-0.484728,True,"[-156.959155383537, -156.97300069417307, -156....","[22.9518861202522, 23.033962489599663, 23.1132...","[-0.15850601318965057, -0.27230734428256403, -...",200001010000 002,20,1,-0.592701
3,3,2000-01-01,-157.279485,25.245133,-0.357612,True,"[-157.737693931696, -157.672992477558, -157.59...","[25.0607392199964, 25.11306391107641, 25.15033...","[-0.10810833887211632, -0.11965485823088562, -...",200001010000 003,17,1,-0.391277
4,4,2000-01-01,-157.825133,24.521705,-0.270798,True,"[-158.294683608633, -158.21303988239845, -158....","[24.521180619821, 24.504563408481385, 24.49381...","[-0.17313183251995282, -0.1924738129886017, -0...",200001010000 004,26,1,-0.359810
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076060,604,2019-12-31,-143.433040,37.663826,-0.185681,True,"[-143.850840243383, -143.78168183541732, -143....","[37.5150790620286, 37.56151716365107, 37.60024...","[-0.05871261071955161, -0.07368344868564174, -...",201912100000 161,22,22,-0.315532
4076061,605,2019-12-31,-142.775877,37.750317,-0.119429,True,"[-142.874272384108, -142.84106378079775, -142....","[37.291241902387, 37.36761672766041, 37.446692...","[-0.03902375593216959, -0.04686758899386807, -...",201912300000 423,2,2,-0.129556
4076062,606,2019-12-31,-141.216740,31.007622,-0.192688,True,"[-141.529817189518, -141.4550550206567, -141.3...","[31.0187962426343, 30.983375812396, 30.9796119...","[-0.06353198294064683, -0.1063841380709802, -0...",201912290000 544,3,3,-0.206966
4076063,607,2019-12-31,-141.283859,28.264153,-0.098530,True,"[-141.622539767485, -141.58178283268998, -141....","[27.9083976838025, 27.98106120292664, 28.05031...","[-0.0546333743178339, -0.0666976217747959, -0....",201912290000 161,3,3,-0.134088


# Calculate translation speeds

We compute translation speeds for every instance of a TRAP trajectory. Therefore we choose all TRAPs that persist for at least three days and average the forward and backward shifted velocity at a current timestamp. The forward/backward shifted velocity is the distance to its succeeding/preceding position divided by the time lapsed between both instances, respectively. This way we deliberately create no velocities at the start and end of a trajectory and do not gain propagation speeds for trajectories of two days lifetime. In turn, we obtain translation speeds at individual TRAP instances which we consider more accurate than taking one one of the shifted velocities or even the full distance travelled by a TRAP divided by the respective lifetime. For this, we iterate through the unique origin IDs, extract subdataframes, calculate speeds and reassign them to the overall dataframe. We assign TRAPS with lifetime below 3 days a translation speed of nan.

In [8]:
# introduce translation speed columns
# use np.nan for TRAPS that have no velocity information since they only existed less than 3 days
# TRAPS which persist at least 3 days will also have nan values at the start and end positions
pd_TRAPS_df['core_U'] = np.nan
pd_TRAPS_df['core_V'] = np.nan

# and get the numpy arrays
core_Us = pd_TRAPS_df.core_U.to_numpy()
core_Vs = pd_TRAPS_df.core_V.to_numpy()

In [9]:
# columns to arrays
origin_IDs = pd_TRAPS_df.origin_ID.to_numpy()
lifetimes = pd_TRAPS_df.lifetime.to_numpy()

# get the unique set of trajectory labels
origin_IDs_unique = np.unique(origin_IDs)
number_of_originIDs = origin_IDs_unique.size

In [10]:
start_timer = time.perf_counter()

In [11]:
# the timedelta between positions, since TRAPS are only tracked over 
# consecutive daily timesteps we can set this a constant value
timedelta = 86400 # in seconds

# only a few iterations for test runs
originID_indices = range(100) if notebook_run else range(number_of_originIDs)

for coriginID_index in originID_indices: # iterate through every trajectory

    corigin_ID_filter = (origin_IDs==origin_IDs_unique[coriginID_index])

    # don't run this on two- or one-day TRAPS, those will remain with nan velocities
    if np.mean(lifetimes[corigin_ID_filter])<3:
        print('skipped    translation speed for originID index ' 
              + str(coriginID_index+1).zfill(len(str(number_of_originIDs))) + '/' + str(number_of_originIDs))
        continue
    
    
    # for the computation we only need a subset of the dataframe
    # this would be faster with numpy arrays but with dataframes it becomes more clear
    # only load the stuff relevant for calculations
    pd_TRAPS_cdf = pd_TRAPS_df.loc[corigin_ID_filter, ['core_lon', 'core_lat']].copy()
    
    # make extra columns with coordinates of the previous timestamp
    pd_TRAPS_cdf['previous_core_lon'] = pd_TRAPS_cdf['core_lon'].shift(1)
    pd_TRAPS_cdf['previous_core_lat'] = pd_TRAPS_cdf['core_lat'].shift(1)

    # calculate the zonal distance to the previous position, positive Eastward in meters, np.cos(radians)
    pd_TRAPS_cdf['dist_to_previous_core_lon'] = (1852*60*np.cos((pd_TRAPS_cdf.core_lat+pd_TRAPS_cdf.previous_core_lat)/2*np.pi/180)*
                                                 (pd_TRAPS_cdf.core_lon-pd_TRAPS_cdf.previous_core_lon))

    # calculate the meridional distance to the previous position, positive Northward in meters
    pd_TRAPS_cdf['dist_to_previous_core_lat'] = 1852*60*(pd_TRAPS_cdf.core_lat - pd_TRAPS_cdf.previous_core_lat)

    # calculate the forward shifted speed over ground and shift it back
    pd_TRAPS_cdf['forward_shifted_core_U'] = pd_TRAPS_cdf.dist_to_previous_core_lon / timedelta # in metres per second
    pd_TRAPS_cdf['backward_shifted_core_U'] = pd_TRAPS_cdf.forward_shifted_core_U.shift(-1)
    
    # the real mean velocity at the TRAP position is the mean of both forward and backward shifted velocities
    # this deliberately creates nan values at the start and end positions
    pd_TRAPS_cdf['core_U'] = (pd_TRAPS_cdf.forward_shifted_core_U + pd_TRAPS_cdf.backward_shifted_core_U) / 2

    
    pd_TRAPS_cdf['forward_shifted_core_V'] = pd_TRAPS_cdf.dist_to_previous_core_lat / timedelta # in metres per second
    pd_TRAPS_cdf['backward_shifted_core_V'] = pd_TRAPS_cdf.forward_shifted_core_V.shift(-1)

    pd_TRAPS_cdf['core_V'] = (pd_TRAPS_cdf.forward_shifted_core_V + pd_TRAPS_cdf.backward_shifted_core_V) / 2
    
    
    # update the respective column arrays of the overall dataframe
    core_Us[corigin_ID_filter] = pd_TRAPS_cdf.core_U.to_numpy()
    core_Vs[corigin_ID_filter] = pd_TRAPS_cdf.core_V.to_numpy()
    
    
    print('calculated translation speed for originID index ' 
          + str(coriginID_index+1).zfill(len(str(number_of_originIDs))) + '/' + str(number_of_originIDs))

    
# and finally update the overall dataframe
pd_TRAPS_df['core_U'] = core_Us
pd_TRAPS_df['core_V'] = core_Vs

# derive velocity magnitude and direction
# pd_TRAPS_df['core_speed'] = np.array(mpcalc.wind_speed(core_Us * units('m/s'), core_Vs * units('m/s')))
# pd_TRAPS_df['core_direction'] = np.array(mpcalc.wind_direction(core_Us * units('m/s'), core_Vs * units('m/s'), convention='to'))

# save memory
del pd_TRAPS_cdf

calculated translation speed for originID index 000001/720391
skipped    translation speed for originID index 000002/720391
calculated translation speed for originID index 000003/720391
calculated translation speed for originID index 000004/720391
calculated translation speed for originID index 000005/720391
skipped    translation speed for originID index 000006/720391
skipped    translation speed for originID index 000007/720391
skipped    translation speed for originID index 000008/720391
skipped    translation speed for originID index 000009/720391
calculated translation speed for originID index 000010/720391
skipped    translation speed for originID index 000011/720391
calculated translation speed for originID index 000012/720391
calculated translation speed for originID index 000013/720391
calculated translation speed for originID index 000014/720391
skipped    translation speed for originID index 000015/720391
calculated translation speed for originID index 000016/720391
calculat

In [12]:
stop_timer = time.perf_counter()
print(f'determined TRAP translation speeds in : {stop_timer - start_timer:0.4f} seconds')

determined TRAP translation speeds in : 7.9232 seconds


In [13]:
# print check
pd_TRAPS_df

,TRAP_ID,time,core_lon,core_lat,core_attraction,ED_TRAP,curve_lons,curve_lats,curve_attractions,origin_ID,lifetime,age,track_s1_peak,core_U,core_V
0,0,2000-01-01,-159.045688,23.400981,-0.625117,True,"[-159.465202832297, -159.40044405039814, -159....","[23.1889083623293, 23.241294361583048, 23.2868...","[-0.31061579015898855, -0.4048254423681092, -0...",200001010000 000,16,1,-0.650914,NaN,NaN
1,1,2000-01-01,-159.503564,24.127516,-0.298961,True,"[-159.159668021201, -159.23795868926015, -159....","[23.8158077645795, 23.84382718973737, 23.89130...","[-0.30928585683127613, -0.24813203636879133, -...",200001010000 001,1,1,-0.298961,NaN,NaN
2,2,2000-01-01,-157.098753,23.351263,-0.484728,True,"[-156.959155383537, -156.97300069417307, -156....","[22.9518861202522, 23.033962489599663, 23.1132...","[-0.15850601318965057, -0.27230734428256403, -...",200001010000 002,20,1,-0.592701,NaN,NaN
3,3,2000-01-01,-157.279485,25.245133,-0.357612,True,"[-157.737693931696, -157.672992477558, -157.59...","[25.0607392199964, 25.11306391107641, 25.15033...","[-0.10810833887211632, -0.11965485823088562, -...",200001010000 003,17,1,-0.391277,NaN,NaN
4,4,2000-01-01,-157.825133,24.521705,-0.270798,True,"[-158.294683608633, -158.21303988239845, -158....","[24.521180619821, 24.504563408481385, 24.49381...","[-0.17313183251995282, -0.1924738129886017, -0...",200001010000 004,26,1,-0.359810,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076060,604,2019-12-31,-143.433040,37.663826,-0.185681,True,"[-143.850840243383, -143.78168183541732, -143....","[37.5150790620286, 37.56151716365107, 37.60024...","[-0.05871261071955161, -0.07368344868564174, -...",201912100000 161,22,22,-0.315532,NaN,NaN
4076061,605,2019-12-31,-142.775877,37.750317,-0.119429,True,"[-142.874272384108, -142.84106378079775, -142....","[37.291241902387, 37.36761672766041, 37.446692...","[-0.03902375593216959, -0.04686758899386807, -...",201912300000 423,2,2,-0.129556,NaN,NaN
4076062,606,2019-12-31,-141.216740,31.007622,-0.192688,True,"[-141.529817189518, -141.4550550206567, -141.3...","[31.0187962426343, 30.983375812396, 30.9796119...","[-0.06353198294064683, -0.1063841380709802, -0...",201912290000 544,3,3,-0.206966,NaN,NaN
4076063,607,2019-12-31,-141.283859,28.264153,-0.098530,True,"[-141.622539767485, -141.58178283268998, -141....","[27.9083976838025, 27.98106120292664, 28.05031...","[-0.0546333743178339, -0.0666976217747959, -0....",201912290000 161,3,3,-0.134088,NaN,NaN


# Export 20-years pickle file

Write the final 20-years TRAPs dataframe to one pickle file.

In [14]:
# create the objects
TRAPS_data = TRAPSdata(vel_product_short, vel_product_long, pd_TRAPS_df)

In [15]:
start_timer = time.perf_counter()

# build the filenames
pkl_TRAPS_exportname = vel_product_short + epsilon_string + '_TRAPS_SPEEDS_0019.pkl'

# save the object as .pkl file                
save_object(TRAPS_data, pkl_TRAPS_exportpath + pkl_TRAPS_exportname)

stop_timer = time.perf_counter()
print('saved ' + pkl_TRAPS_exportname + f' in: {stop_timer - start_timer:0.4f} seconds')

saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_0019.pkl in: 68.1137 seconds


# Export yearly pickle files

For every year, copy the respective TRAP objects to a current dataframe and export this dataframe as pkl file.  
The yearly files are more handy for analysis.

In [16]:
for year in years:

    start_timer = time.perf_counter()
    
    # filter dataframe for objects of the current year
    cyear_filter = (pd_TRAPS_df.time.dt.year == int(year))
    
    # extract TRAP objects of the current year to a current dataframe
    pd_TRAPS_cdf = pd_TRAPS_df[cyear_filter].copy()
    
    # reset the index to avoid errors later
    pd_TRAPS_cdf.reset_index(drop=True, inplace=True)
    
    # construct the export file name
    pkl_TRAPS_exportname = vel_product_short + epsilon_string + '_TRAPS_SPEEDS_' + year + '.pkl'
    
    # overwrite any other export path, yearly files always go here
    pkl_TRAPS_exportpath = 'export_pkl/' + vel_product_short + '/20XX/'
    
    # create the object
    TRAPS_data = TRAPSdata(vel_product_short, vel_product_long, pd_TRAPS_cdf)
    
    # save the object as .pkl file
    save_object(TRAPS_data, pkl_TRAPS_exportpath + pkl_TRAPS_exportname)


    stop_timer = time.perf_counter()
    print('saved ' + pkl_TRAPS_exportname + f' in: {stop_timer - start_timer:0.1f} seconds')
    
    

saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2000.pkl in: 6.3 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2001.pkl in: 2.9 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2002.pkl in: 2.8 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2003.pkl in: 2.7 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2004.pkl in: 3.0 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2005.pkl in: 6.0 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2006.pkl in: 2.8 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2007.pkl in: 2.7 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2008.pkl in: 2.6 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2009.pkl in: 2.7 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2010.pkl in: 6.2 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2011.pkl in: 2.8 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2012.pkl in: 2.8 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2013.pkl in: 2.9 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2014.pkl in: 2.8 seconds
saved MULTIOBS_24HI_e025_TRAPS_SPEEDS_2015.pkl in: 6.1 

In [17]:
# print check
pd_TRAPS_cdf

,TRAP_ID,time,core_lon,core_lat,core_attraction,ED_TRAP,curve_lons,curve_lats,curve_attractions,origin_ID,lifetime,age,track_s1_peak,core_U,core_V
0,0,2019-01-01,-158.458364,24.059164,-0.351214,True,"[-158.38374860392, -158.41960961414034, -158.4...","[23.7460527870143, 23.820894383255276, 23.9024...","[-0.11968049575536073, -0.21891474142828818, -...",201812190000 218,29,14,-0.405814,NaN,NaN
1,1,2019-01-01,-159.535267,23.507555,-0.390133,True,"[-159.875, -159.8034298403529, -159.7368731187...","[23.1939143562609, 23.23653382035548, 23.28660...","[-0.3532991787329774, -0.2957999519786219, -0....",201812310000 002,5,2,-0.401339,NaN,NaN
2,2,2019-01-01,-158.338473,23.286057,-0.166323,True,"[-158.784961092821, -158.70266004509017, -158....","[23.3014697575335, 23.313026128099303, 23.3038...","[-0.052507955074994395, -0.08242565084126094, ...",201812220000 001,15,11,-0.320469,NaN,NaN
3,3,2019-01-01,-156.391062,24.198454,-0.398398,True,"[-155.951947969633, -156.02314086867204, -156....","[24.0225231429828, 24.065434766636216, 24.0984...","[-0.1495098399019887, -0.22146161183715224, -0...",201812090000 028,41,24,-0.466162,NaN,NaN
4,4,2019-01-01,-157.477970,22.670770,-0.205526,False,"[-157.738275005969, -157.6572390964539, -157.5...","[22.625, 22.64363071744845, 22.658392278681696...","[-0.12647874700806655, -0.16345862837512035, -...",201901010000 004,1,1,-0.205526,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211192,604,2019-12-31,-143.433040,37.663826,-0.185681,True,"[-143.850840243383, -143.78168183541732, -143....","[37.5150790620286, 37.56151716365107, 37.60024...","[-0.05871261071955161, -0.07368344868564174, -...",201912100000 161,22,22,-0.315532,NaN,NaN
211193,605,2019-12-31,-142.775877,37.750317,-0.119429,True,"[-142.874272384108, -142.84106378079775, -142....","[37.291241902387, 37.36761672766041, 37.446692...","[-0.03902375593216959, -0.04686758899386807, -...",201912300000 423,2,2,-0.129556,NaN,NaN
211194,606,2019-12-31,-141.216740,31.007622,-0.192688,True,"[-141.529817189518, -141.4550550206567, -141.3...","[31.0187962426343, 30.983375812396, 30.9796119...","[-0.06353198294064683, -0.1063841380709802, -0...",201912290000 544,3,3,-0.206966,NaN,NaN
211195,607,2019-12-31,-141.283859,28.264153,-0.098530,True,"[-141.622539767485, -141.58178283268998, -141....","[27.9083976838025, 27.98106120292664, 28.05031...","[-0.0546333743178339, -0.0666976217747959, -0....",201912290000 161,3,3,-0.134088,NaN,NaN


## End sound

In [18]:
# measure the computation time for the entire script
stop_script_timer = time.perf_counter()
print(f'overall computation time: {stop_script_timer - start_script_timer:0.3f} seconds')

overall computation time: 182.908 seconds


In [19]:
#https://gist.github.com/tamsanh/a658c1b29b8cba7d782a8b3aed685a24

framerate = 4410
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
# G-Dur
#audio_data = np.sin(2*np.pi*391*t) + np.sin(2*np.pi*493*t) + np.sin(2*np.pi*587*t)
# D-Dur
audio_data = np.sin(2*np.pi*293*t) + np.sin(2*np.pi*369*t) + np.sin(2*np.pi*440*t)
Audio(audio_data, rate=framerate, autoplay=True)